In [1]:
import torch
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import torchtext

In [2]:
##Change working directory to talk-berty-to-me root
import os
os.chdir("D:/University/Projects/AML/talk-berty-to-me")

In [ ]:
# data_gutenberg = pd.read_csv('data/books_and_genres.csv')
# dataset_train = data_gutenberg.sample(frac=0.6, random_state=0)
# dataset_val = data_gutenberg.drop(dataset_train.index).sample(frac=0.5, random_state=0)
# dataset_test = data_gutenberg.drop(dataset_train.index).drop(
#     dataset_val.index)
# dataset_train.to_parquet('data/datasets/train.parquet', index=False)
# dataset_test.to_parquet('data/datasets/test.parquet', index=False)
# dataset_val.to_parquet('data/datasets/val.parquet', index=False)
# data_gutenberg.sample(20).to_parquet('data/datasets/dev.parquet', index=False)

In [ ]:
#Add code to switch between datasets here

In [ ]:
# dev_data = pd.read_parquet('data/datasets/dev.parquet')

In [3]:
raw_data = pd.read_csv('data/books_and_genres_eng.csv')

In [4]:
raw_data.head()

,Unnamed: 0,title,text,genres,language_code
0,31,the power and the glory,"Produced by Juliet Sutherland, Sjaani and PG D...","{'literary-fiction', 'christian', 'history', '...",eng
1,32,paradise,Produced by Judith Smith and Natalie Salter\n\...,"{'literary-fiction', 'mythology', 'historical-...",eng
2,88,sonnets,"Produced by Paul Murray, Rénald Lévesque and t...","{'read-for-school', 'poetry', '20th-century', ...",eng
3,88,sonnets,"Produced by Paul Murray, Rénald Lévesque and t...","{'read-for-school', 'poetry', '20th-century', ...",eng
4,147,persuasion,Produced by Sharon Partridge and Martin Ward. ...,"{'romance', 'literary-fiction', 'classics', 'h...",eng


In [16]:
dev_data = raw_data.sample(20)
dev_data.drop(columns=['Unnamed: 0'], inplace=True)

###Creating a validation set
val_data = raw_data.sample(5)
val_data.drop(columns=['Unnamed: 0'], inplace=True)

In [17]:
dev_data.head()

,title,text,genres,language_code
143,phaedrus,Produced by Sue Asscher\n\n\n\n\n\nPHAEDRUS\n\...,"{'history', 'classics', 'mythology', 'philosop...",eng
158,the picture of dorian gray,"Produced by David Clarke, Chuck Greif and the ...","{'literary-fiction', 'historical-fiction', 'hi...",en-US
405,don quixote,Produced by David Widger\n\n\n\n\n\nDON QUIXOT...,"{'adventure', 'literary-fiction', 'history', '...",en-US
81,the jungle,"Produced by David Meltzer, Christy Phillips, S...","{'literary-fiction', 'history', 'historical-fi...",eng
179,alice in wonderland,"Produced by Chuck Greif, Jason Isbell and The ...","{'adventure', 'middle-grade', 'classics', 'fan...",eng


In [18]:
val_data.head()

,title,text,genres,language_code
378,robinson crusoe,Produced by Bruce W. Miller\n\n\n\n\n\n\n\n\nR...,"{'adventure', 'roman', 'classics', 'novels', '...",eng
385,frankenstein,"Produced by Judith Boss, Christy Phillips, Lyn...","{'horror', 'mystery', 'classics', 'supernatura...",en-US
76,macbeth,Produced by Delphine Lettau\n\n\n\n\nThis book...,"{'history', 'historical-fiction', 'historical'...",eng
169,oedipus the king,Produced by Christos Alexandridis\n\n\n\n\n\nT...,"{'history', 'historical', 'classics', 'high-sc...",eng
145,the girl with the golden eyes,"Produced by John Bickers, and Dagny\n\n\n\n\n ...","{'romance', 'literary-fiction', 'drama', 'roma...",eng


In [19]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
def clean_text(text):
    '''
    Function to clean text of books. Removes email addresses, new lines, html tags, and extra spaces.

    Input: Text (String)
    Output: Cleaned Text (String)
    '''
    cleaned_text = text.lower()
    cleaned_text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' ', text)
    cleaned_text = re.sub(r'^.*?(?=\n\n\n)', ' ', cleaned_text, flags=re.DOTALL)
    cleaned_text = re.sub(r'<a\s+(?:[^>]*?\s+)?href="([^"]*)"[^>]*>.*?</a>', ' ', cleaned_text, flags=re.DOTALL)
    cleaned_text = re.sub(r'\n', ' ', cleaned_text)
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    cleaned_text = re.sub(r'[^\w\s.?!]', ' ', cleaned_text)
    cleaned_text = re.sub(r' +', ' ', cleaned_text)
    return cleaned_text

def remove_first_row(group):
    return group.iloc[1:]

def lowercase(text):
    return text.lower()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\setul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\setul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
dev_data['cleaned_text'] = dev_data['text'].apply(lambda x:clean_text(x))
dev_data.loc[:,'genres'] = dev_data.loc[:,'genres'].apply(lambda x:clean_text(x))
dev_data.loc[:,'sentences'] = dev_data.loc[:,'cleaned_text'].apply(lambda x: nltk.tokenize.sent_tokenize(str(x)))
dev_data = dev_data.explode('sentences')
dev_data.loc[:,'sentences'] = dev_data.loc[:,'sentences'].apply(lambda x:lowercase(x))
dev_data.reset_index(drop=True, inplace=True)
dev_data['label_sentences'] = dev_data.groupby('title')['sentences'].shift(-1)
dev_data = dev_data.dropna(subset=['label_sentences'])
dev_data = dev_data.loc[:,['title', 'genres','sentences', 'label_sentences']]

In [21]:
val_data['cleaned_text'] = val_data['text'].apply(lambda x:clean_text(x))
val_data.loc[:,'genres'] = val_data.loc[:,'genres'].apply(lambda x:clean_text(x))
val_data.loc[:,'sentences'] = val_data.loc[:,'cleaned_text'].apply(lambda x: nltk.tokenize.sent_tokenize(str(x)))
val_data = val_data.explode('sentences')
val_data.loc[:,'sentences'] = val_data.loc[:,'sentences'].apply(lambda x:lowercase(x))
val_data.reset_index(drop=True, inplace=True)
val_data['label_sentences'] = val_data.groupby('title')['sentences'].shift(-1)
val_data = val_data.dropna(subset=['label_sentences'])
val_data = val_data.loc[:,['title', 'genres','sentences', 'label_sentences']]

In [22]:
dev_data = dev_data.loc[:,['title', 'genres','sentences', 'label_sentences']]
dev_data.head()

,title,genres,sentences,label_sentences
0,phaedrus,history classics mythology philosophy fiction...,phaedrus by plato translated by benjamin jowe...,the phaedrus is closely connected with the sym...
1,phaedrus,history classics mythology philosophy fiction...,the phaedrus is closely connected with the sym...,the two dialogues together contain the whole p...
2,phaedrus,history classics mythology philosophy fiction...,the two dialogues together contain the whole p...,but in the phaedrus and symposium love and phi...
3,phaedrus,history classics mythology philosophy fiction...,but in the phaedrus and symposium love and phi...,the spiritual and emotional part is elevated i...
4,phaedrus,history classics mythology philosophy fiction...,the spiritual and emotional part is elevated i...,whether the subject of the dialogue is love or...


In [23]:
val_data = val_data.loc[:,['title', 'genres','sentences', 'label_sentences']]
val_data.head()

,title,genres,sentences,label_sentences
0,robinson crusoe,adventure roman classics novels fiction adult...,robinson crusoe in words of one syllable by m...,the production of a book which is adapted to t...
1,robinson crusoe,adventure roman classics novels fiction adult...,the production of a book which is adapted to t...,the nature of the work seems to be sufficientl...
2,robinson crusoe,adventure roman classics novels fiction adult...,the nature of the work seems to be sufficientl...,but although as far as the subject matter is c...
3,robinson crusoe,adventure roman classics novels fiction adult...,but although as far as the subject matter is c...,the deep interest which de foe s story has nev...
4,robinson crusoe,adventure roman classics novels fiction adult...,the deep interest which de foe s story has nev...,it should be stated that exceptions to the rul...


In [24]:
dev_data.loc[1905, 'sentences']

'i don t know which to follow.'

In [25]:
#Building vocabulary
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
vocab_iter = iter(dev_data.loc[:,'sentences'] + dev_data.loc[:,'title'] + dev_data.loc[:,'genres'])
def yield_tokens(train_iter):
    for text in train_iter:
        if not isinstance(text, str):
            if type(text) == list:
                for t in text:
                    yield tokenizer(t)
            continue
        yield tokenizer(text)

In [26]:
vocab = build_vocab_from_iterator(
    yield_tokens(vocab_iter), specials=["<unk>", "<pad>", "<BOS>", "<EOS>"], min_freq=50)
vocab.set_default_index(vocab["<unk>"])

In [27]:
len(vocab)

3284

In [28]:
VECTOR_CACHE_DIR = '/Users/setul/mlpp23/.vector_cache'
glove = torchtext.vocab.GloVe('6B', cache=VECTOR_CACHE_DIR)
glove_vectors = glove.get_vecs_by_tokens(vocab.get_itos())

In [29]:
dev_data.iloc[2480:2486]

,title,genres,sentences,label_sentences
2481,the picture of dorian gray,literary fiction historical fiction historica...,he exclaimed but at that moment the duke of be...,he is gone murmured sibyl sadly.
2482,the picture of dorian gray,literary fiction historical fiction historica...,he is gone murmured sibyl sadly.,i wish you had seen him.
2483,the picture of dorian gray,literary fiction historical fiction historica...,i wish you had seen him.,i wish i had for as sure as there is a god in ...
2484,the picture of dorian gray,literary fiction historical fiction historica...,i wish i had for as sure as there is a god in ...,she looked at him in horror.
2485,the picture of dorian gray,literary fiction historical fiction historica...,she looked at him in horror.,he repeated his words.
2486,the picture of dorian gray,literary fiction historical fiction historica...,he repeated his words.,they cut the air like a dagger.


In [30]:
import torch.nn as nn
import torch.nn.functional as F
IS_CUDA = torch.cuda.is_available()
if IS_CUDA:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [31]:
from torch.nn.utils.rnn import pad_sequence
def collate_batch(batch):
    titles, genres, sentences, label_sentences = zip(*batch)
    #Add a separator tag between title and genre
    context = [tokenizer(g) + ['<pad>'] + tokenizer(t) + ['<pad>'] +
               ['<BOS>'] + tokenizer(s) + ['<EOS>'] for t, g,
                s in zip(titles, genres, sentences)]
    label_sentence = [['<BOS>'] + tokenizer(s) + ['<EOS>'] for s in label_sentences]
    label_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in label_sentence],
                                    padding_value=vocab['<pad>'], batch_first=True)
    encoder_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in context],
                                 padding_value=vocab['<pad>'], batch_first=True)
    return encoder_tensor, label_tensor

In [32]:
#From HW4
from torch.utils.data import Sampler
class BatchSequentialSampler(Sampler):
    r"""Samples batches, s.t. the ith elements of each batch are sequential.

    Args:
        data_source (Dataset): dataset to sample from
    """

    def __init__(self, data_source, batch_size):
        self.data_source = data_source
        self.batch_size = batch_size
        
    def __iter__(self):
        num_batches = len(self.data_source)//self.batch_size
        for i in range(num_batches):
            for j in range(self.batch_size):
                yield(j * num_batches + i)

    def __len__(self):
        return (len(self.data_source)//self.batch_size) * self.batch_size

In [33]:
batch_sampler = BatchSequentialSampler(dev_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']], 64)

In [34]:
batch_dataloader = torch.utils.data.DataLoader(dev_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']].values,
                                                   batch_size=64, collate_fn=collate_batch, sampler=batch_sampler)

In [36]:
val_sampler = BatchSequentialSampler(val_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']], 64)

In [37]:
val_dataloader = torch.utils.data.DataLoader(val_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']].values,
                                                   batch_size=64, collate_fn=collate_batch, sampler=val_sampler)

In [38]:
for idx, (context_tensor, label_tensor) in enumerate(batch_dataloader):
    print(context_tensor.shape)
    print(label_tensor.shape)
    break

torch.Size([64, 111])
torch.Size([64, 88])


In [39]:
class BiRNN_encoder(nn.Module):
    def __init__ (self, embedding_dim, hidden_dim,
                  vocab_size, num_layers=2, type_rnn = 'LSTM', bidirectional = True,
                  dropout = 0.3, pad_idx = 0):
        super(BiRNN_encoder, self).__init__()
        self.rnns = []
        # self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx).to(device)
        self.embedding = nn.Embedding.from_pretrained(glove_vectors).to(device)
        hidden_size = hidden_dim * 2 if bidirectional else hidden_dim
        input_size = embedding_dim
        for _ in range(num_layers):
            if type_rnn == 'LSTM':
                rnn = nn.LSTM(input_size, hidden_size, 1, dropout = dropout,
                               bidirectional = bidirectional, batch_first=True).to(device)
            elif type_rnn == 'GRU':
                rnn = nn.GRU(input_size, hidden_size, 1, dropout = dropout,
                              bidirectional = bidirectional, batch_first=True).to(device)
            self.rnns.append(rnn)
            input_size = hidden_size*2 if bidirectional else hidden_size
        self.rnns = nn.ModuleList(self.rnns).to(device)
        self.dropout = nn.Dropout(dropout).to(device)
        self.type_rnn = type_rnn
        self.bidirectional = bidirectional
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

    def forward(self, input, hidden = None):
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)
        rnn_input = embedded
        for idx, rnn in enumerate(self.rnns):
            output, hidden_output = rnn(rnn_input, hidden)
            hidden = hidden_output
            rnn_input = output
        if self.type_rnn == 'LSTM' and self.bidirectional:
            hidden_state = torch.cat((hidden[0][-2,:,:], hidden[0][-1,:,:]), dim = 1).to(device)
            cell = torch.cat((hidden[1][-2,:,:], hidden[1][-1,:,:]), dim = 1).to(device)
            hidden = (hidden_state, cell)
        elif self.type_rnn == 'GRU' and self.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1).to(device)
        return hidden

In [40]:
encoder = BiRNN_encoder(300, 600, len(vocab), num_layers=2, type_rnn = 'LSTM',
                         bidirectional = True, dropout = 0.3, pad_idx = vocab['<pad>']).to(device)

c:\Users\setul\miniconda3\envs\talk-berty\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [41]:
for i, batch in enumerate(batch_dataloader):
    context_tensor, label_tensor = batch
    context_tensor, label_tensor = context_tensor.to(device), label_tensor.to(device)
    hidden,cell = encoder(context_tensor)
    print(hidden.shape)
    print(cell.shape)
    break

torch.Size([64, 2400])
torch.Size([64, 2400])


In [42]:
class BiRNN_decoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers, vocab_dim, dropout = 0.3):
        super(BiRNN_decoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.vocab_dim = vocab_dim
        self.dropout = nn.Dropout(dropout).to(device)
        # self.embedding = nn.Embedding(vocab_dim, embedding_dim).to(device)
        self.embedding = nn.Embedding.from_pretrained(glove_vectors).to(device)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout = dropout).to(device)
        self.fc_out = nn.Linear(hidden_dim, vocab_dim).to(device)


    def forward(self, input, hidden, context):
        embedded = self.dropout(self.embedding(input))
        outputs, (hidden,context) = self.rnn(embedded, (hidden, context))
        predictions = self.fc_out(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden, context
    


In [43]:
decoder = BiRNN_decoder(300, 2400, 1, len(vocab), dropout = 0.3).to(device)

In [44]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, src, trg, teacher_ratio = 0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = len(vocab)
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
        hidden, context = self.encoder(src)
        hidden = hidden.detach()
        context = context.detach()
        dec_input = trg[:, 0]
        dec_input = dec_input.unsqueeze(0)
        hidden = hidden.unsqueeze(0)
        context = context.unsqueeze(0)
        for t in range(1, trg_len):
            output, hidden, context = self.decoder(dec_input, hidden, context)
            outputs[t] = output
            top1 = output.argmax(1)
            dec_input = trg[:,t] if np.random.random() < teacher_ratio else top1
            dec_input = dec_input.unsqueeze(0)
        return outputs

In [45]:
USE_CUDA = True

In [46]:
def evaluate(model, data_loader):
    '''
    Evaluate the model on the given data.
    '''

    model.eval()
    it = iter(data_loader)
    total_count = 0. # Number of target words seen
    total_loss = 0. # Loss over all target words
    with torch.no_grad():
        # No gradients need to be maintained during evaluation
        # There are no hidden tensors for the first batch, and so will default to zeros.
        hidden = None 
        for i, batch in enumerate(it):
            text, target = batch
            if USE_CUDA:
                text, target = text.cuda(), target.cuda()
            output = model(text, target).to(device)
            mask = (target != PAD_IDX)
            ntotal = mask.sum()
            loss = loss_fn(output.view(-1, output.size(-1)), target.view(-1))
            
            total_count += ntotal
            total_loss += loss.item()*ntotal
                
    loss = total_loss / total_count
    model.train()
    return loss

In [ ]:
##ChatGPT's improvement
def evaluate(model, data_loader):
    model.eval()
    total_loss = 0.0
    total_count = 0  # Total number of non-<pad> tokens

    with torch.no_grad():
        for text, target in data_loader:
            if USE_CUDA:
                text, target = text.cuda(), target.cuda()

            output = model(text, target)
            mask = (target != PAD_IDX)  # Create a mask for non-pad tokens
            ntotal = mask.sum().item()  # Sum the mask values to get the total number of non-<pad> tokens

            # Apply mask to filter out loss contributions from <pad> tokens and compute the loss
            loss = loss_fn(output.view(-1, output.size(-1)), target.view(-1))
            loss = loss.masked_select(mask.view(-1)).mean()  # Only consider non-<pad> tokens and compute mean loss
            
            total_loss += loss.item() * ntotal  # Accumulate the total loss
            total_count += ntotal  # Accumulate the total count of non-<pad> tokens

    average_loss = total_loss / total_count  # Compute the average loss over all non-<pad> tokens
    model.train()
    return average_loss

In [47]:
LOG_FILE = "language-model.log"
GRAD_CLIP = 1.
NUM_EPOCHS = 10
PAD_IDX = vocab['<pad>']

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if h is None:
        return None
    elif isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
    
    
seq_model = Seq2Seq(encoder, decoder).to(device)
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_IDX, reduction = 'sum') ## Used instead of NLLLoss.
learning_rate = 0.001
optimizer = torch.optim.Adam(seq_model.parameters(), lr=learning_rate)
val_losses = []
best_model = None
for epoch in range(NUM_EPOCHS):
    seq_model.train()
    it = iter(batch_dataloader)
    for i, batch in enumerate(it):
        data, target = batch
        data, target = data.to(device), target.to(device)
        output = seq_model(data, target).to(device)
        output = output[1:].reshape(-1, output.shape[2])
        target = target[:,1:].reshape(-1)
        optimizer.zero_grad()
        mask = (target != PAD_IDX)
        ntotal = mask.sum()
        loss = loss_fn(output, target)
        loss = loss / ntotal
        loss.backward()
        torch.nn.utils.clip_grad_norm_(seq_model.parameters(), GRAD_CLIP)
        optimizer.step()


        if i % 1000 == 0:
            print("epoch", epoch, "iter", i, "loss", loss.item())

        if i % 10000 == 0:
            val_loss = evaluate(seq_model, val_dataloader)
            with open(LOG_FILE, "a") as fout:
                #print("epoch: {}, iteration: {}, perplexity: {}".format(epoch, i, np.exp(val_loss)))
                fout.write("epoch: {}, iteration: {}, perplexity: {}\n".format(epoch, i, val_loss))

            # Save the model if the validation loss is the minimum so far
            if len(val_losses) == 0 or val_loss < min(val_losses):
                print("best model, val loss: ", val_loss)
                # #best_model = copy.deepcopy(model)
                # best_model = type(seq_model)(vocab_size, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)
                # if USE_CUDA:
                #     best_model = best_model.cuda()
                # best_model.load_state_dict(model.state_dict())

                # with open(PATH + "lm-best.th", "wb") as fout:
                #     torch.save(best_model.state_dict(), fout)
            else:
                learning_rate /= 4.
                optimizer = torch.optim.Adam(seq_model.parameters(), lr=learning_rate)
            val_losses.append(val_loss)


epoch 0 iter 0 loss 8.098148345947266


In [ ]:
# ##From other sources
# epoch_loss = 0.0
# num_epochs = 10
# best_loss = 999999
# best_epoch = -1
# sentence1 = "Hello I am starting"
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.CrossEntropyLoss(ignore_index = vocab['<pad>'])
# ts1 = []
# for epoch in range(num_epochs):
#   print("Epoch - {} / {}".format(epoch+1, num_epochs))
#   model.train(True)
#   for batch_idx, batch in enumerate(batch_dataloader):
#     input , target = batch
#     input, target = input.to(device), target.to(device)
#     output = model(input, target).to(device)
#     output = output[1:].reshape(-1, output.shape[2])
#     target = target[:,1:].reshape(-1)
    
#     optimizer.zero_grad()
#     loss = criterion(output, target)
#     loss.backward()

#     # Clip gradient >1 to prevent exploding gradients
#     torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

#     # Update the weights values using the gradients we calculated using bp 
#     optimizer.step()
#     #step += 1
#     epoch_loss += loss.item()
#     #writer.add_scalar("Training loss", loss, global_step=step)

#   if epoch_loss < best_loss:
#     best_loss = epoch_loss
#     best_epoch = epoch
#     if ((epoch - best_epoch) >= 3):
#       print("no improvement in 3 epochs, break")
#       break
#   print("Epoch_Loss - {}".format(loss.item()))
#   print()
  
# print(epoch_loss / len(batch_dataloader))

In [ ]:
torch.save(seq_model.state_dict(), 'seq_model_functional.pt')

In [ ]:
model = torch.load('model.pt')

In [ ]:
def generate_text(model, input_genre, input_title, input_text, max_length=10):
    model.eval()

    # Tokenize and prepare input text
    input_tokens = (tokenizer(input_genre) + ['<pad>'] + tokenizer(input_title) + ['<pad>'] +
    ['<BOS>'] + tokenizer(input_text) + ['<EOS>'])
    input_indices = vocab.lookup_indices(input_tokens)
    input_tensor = torch.tensor([input_indices], dtype=torch.long, device=device)

    with torch.no_grad():
        hidden, context = model.encoder(input_tensor)
    decoder_input = torch.tensor([[vocab['<BOS>']]], device=device)  
    output_indices = []
    context = context.unsqueeze(0)
    hidden = hidden.unsqueeze(0)

    for _ in range(max_length):
        with torch.no_grad():
            # print("decoder_shape", decoder_input.shape)
            # print("hidden_shape", hidden.shape)
            # print("context_shape", context.shape)
            output, hidden, context = model.decoder(decoder_input, hidden, context)
            output_probabilities = output.squeeze().exp().to(device)
            top1 = torch.multinomial(output_probabilities, 1)[0]
            #top1 = torch.argmax(output)
            # print(top1)
            # print("top1 shape", top1.shape)
            # print(vocab.lookup_token(top1.item()))
            #print(vocab.lookup_indices(list(top1)))
            # print(top1)
            if top1.item() == vocab['<EOS>']:
                break
            output_indices.append(top1.item())

            decoder_input = torch.tensor([[top1.item()]], device=device)  
    output_tokens = [vocab.lookup_token(index) for index in output_indices]
    return ' '.join(output_tokens)


In [ ]:
vocab.lookup_token(30)

In [ ]:
input_title = "autobiography"
input_genre = "crime"
input_sentence = "The room was close."
generated_text = generate_text(seq_model, input_genre, input_title, input_sentence, max_length=50)
print(generated_text)

In [ ]:
for param in seq_model.parameters():
    print("param", param.grad)